In [4]:
using JuMP, Gurobi, CSV, DataFrames, Statistics, LinearAlgebra
model = Model(optimizer_with_attributes(Gurobi.Optimizer, 
    "TimeLimit" => 60,
    "MIPGap" => 0.01  # 1% optimality gap
))

fixed_cost = 200000.0
variable_cost = 125.0
transportation_cost = 3.14
delta = 0.95

plant_details_df = CSV.read("cleaned_plant_locations.csv", DataFrame; header=false)
distance_df = CSV.read("distance_mtx.csv", DataFrame; header=false)

# making statistics to replace broken data
valid_values = Float64[]
for i in 1:size(distance_df, 1)
    for j in 1:size(distance_df, 2)
        val = distance_df[i, j]
        if val isa AbstractString
            parsed = tryparse(Float64, string(val))
            if !isnothing(parsed)
                push!(valid_values, parsed)
            end
        else
            push!(valid_values, Float64(val))
        end
    end
end

# Compute statistics on valid values
max_valid = isempty(valid_values) ? 1_000_000.0 : maximum(valid_values)
mean_valid = isempty(valid_values) ? 1_000.0 : mean(valid_values)

# Convert to numeric matrix with error handling
for i in 1:size(distance_df, 1)
    for j in 1:size(distance_df, 2)
        val = distance_df[i, j]
        if val isa AbstractString
            str_val = string(val)
            if str_val == "NET_ERR"
                distance_mtx_raw[i, j] = max_valid * 100.0
            else
                # Try to parse other strings
                parsed = tryparse(Float64, str_val)
                if isnothing(parsed)
                    # If still can't parse, use mean of valid values
                    distance_mtx_raw[i, j] = mean_valid
                else
                    distance_mtx_raw[i, j] = parsed
                end
            end
        else
            distance_mtx_raw[i, j] = Float64(val)
        end
    end
end

# meters to miles
distance_mtx_raw .*= 0.000621371

@variable(model, x[1:95, 1:12], Bin)           # loc_made[i,t]
@variable(model, y[1:95] >= 0)                 # loc_size[i]
@variable(model, z[1:338, 1:95, 1:12] >= 0)    # solar_to_loc[s,i,t]
@variable(model, e[1:95], Bin)                 # facility_exists[i]
@variable(model, b[1:95, 1:12], Bin)           # started_by_year[i,t]
@variable(model, a[1:95, 1:12], Bin)           # year_active[i,t]
@variable(model, w[1:95, 1:12] >= 0)           # bilinear term x[i,t]*y[i]

# Facility existence constraints
@constraint(model, [i=1:95, t=1:12], e[i] >= x[i,t])
@constraint(model, [i=1:95], e[i] <= sum(x[i,t] for t in 1:12))

# Capacity constraints
M = 1000000
@constraint(model, [i=1:95], y[i] <= M * e[i])
@constraint(model, [i=1:95], y[i] >= e[i])

# First time location opens
@constraint(model, [i=1:95], b[i,1] == x[i,1])
@constraint(model, [i=1:95, t=2:12], b[i,t] >= b[i,t-1])
@constraint(model, [i=1:95, t=2:12], b[i,t] >= x[i,t])
@constraint(model, [i=1:95, t=2:12], b[i,t] <= b[i,t-1] + x[i,t])

# Enforcing discount Factor
@constraint(model, [i=1:95], sum(a[i,t] for t in 1:12) == e[i])
@constraint(model, [i=1:95, t=1:12], a[i,t] <= b[i,t])
@constraint(model, [i=1:95, t=1:11], a[i,t] >= b[i,t] - b[i,t+1])
@constraint(model, [i=1:95], a[i,12] >= b[i,12])

# Linearized x[i,t] * y[i]
@constraint(model, [i=1:95, t=1:12], w[i,t] <= M * x[i,t])
@constraint(model, [i=1:95, t=1:12], w[i,t] <= y[i])
@constraint(model, [i=1:95, t=1:12], w[i,t] >= y[i] - M * (1 - x[i,t]))


year_exps_raw =  plant_details[:, 4]
tons_waste   = plant_details[:, 5].*200.0

year_exps = []
#tons_waste = []
for i in 1:338
    push!(year_exps, year_exps_raw[i] - 2007) #first expires in 2008
 #   push!(tons_waste, plant_details[i,5] * 200) #1 MW at average (not optimal) running is estimated to generate 200 tons waste
end


for s in 1:338
    for t in 1:12
        if t != year_exps[s]
            @constraint(model, [j=1:95], z[s,j,t] == 0)
        else
            @constraint(model, tons_waste[s] == sum(z[s,j,t] for j in 1:95))
            @constraint(model, [j=1:95], z[s,j,t] <= y[j])
        end
    end
end

discount_factors = [delta^t for t in 1:12]

@objective(model, Min,
    sum(
        # Fixed cost with discount
        sum(a[i,t] * discount_factors[t] * fixed_cost for t in 1:12) +
        
        # Variable cost with annual discounting
        sum(
            w[i,t] * (delta^t) * variable_cost +
            sum(z[s,i,t] * distance_mtx_raw[i,s] * transportation_cost 
                for s in 1:338)
            for t in 1:12
        )
        for i in 1:95
    )
)

optimize!(model)
termination_status(model)



Set parameter Username
Set parameter LicenseID to value 2703402
Academic license - for non-commercial use only - expires 2026-09-04
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.01

Optimize a model with 396108 rows, 390070 columns and 475760 nonzeros
Model fingerprint: 0x9648ffb8
Variable types: 386555 continuous, 3515 integer (3515 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+06]
Presolve removed 353305 rows and 353305 columns
Presolve time: 0.21s
Presolved: 42803 rows, 36765 columns, 12

TIME_LIMIT::TerminationStatusCode = 12